In [1]:
import pandas as pd
import numpy as np
from glob import glob
import importlib
import agm_util.dataloader as dataloader
importlib.reload(dataloader)
import agm_util.agmlearn as agmlearn
importlib.reload(agmlearn)
from multiprocessing import Pool
import pickle
from ipywidgets import FloatProgress
from multiprocessing import Pool
import itertools

PIDs = dataloader.GetPIDs("Data/Results with expert times (2).xlsx",range(2,62))

In [ ]:
import matplotlib.pyplot as plt
#%pylab inline

In [ ]:
LearningVel = dataloader.GetVelocityData('Data/VelocityRoom/Learning_deltaT',PIDs)
PracticeVel = dataloader.GetVelocityData('Data/VelocityRoom/Practice_deltaT',PIDs)
LearningPos = dataloader.GetPositionData('Data/Position/motionLearning/',PIDs)
PracticePos = dataloader.GetPositionData('Data/Position/motionPractice/',PIDs)

LVfea = dataloader.ProcessToFeaturized(LearningVel)
PVfea = dataloader.ProcessToFeaturized(PracticeVel)
LPfea = dataloader.ProcessToFeaturized(LearningPos)
PPfea = dataloader.ProcessToFeaturized(PracticePos)

In [ ]:
pickle.dump((LVfea, PVfea, LPfea, PPfea),open('Pickles/FeatureVectors/cleananalysis.pkl','wb'))
#LVfea,PVfea,LPfea,PPfea = pickle.load(open('Pickles/FeatureVectors/cleananalysis.pkl','rb'))

In [ ]:
LVfea,PVfea,LPfea,PPfea = pickle.load(open('Pickles/FeatureVectors/cleananalysis.pkl','rb'))

## train Leave-One-Out position classifiers

In [ ]:
inoutpairs = dataloader.GetOrLoadLeaveOneOuts('start.pkl')

In [ ]:
##Here is what training leave-one-out classifierss looks like single-threaded.
#for i,(dftrain,dfpred) in enumerate(inoutpairs):
#    agmlearn.BatteryTest('kNN',[3],'Pos',(i,(dftrain,dfpred)))
#    break

In [ ]:
for k in range(3,10):
    with Pool(8) as p:
        p.starmap(agmlearn.BatteryTest,itertools.product(['kNN'],[[k]],['Normpos'],enumerate(agmlearn.NormalizeList(inoutpairs))))

In [ ]:
with Pool(8) as p:
    p.starmap(agmlearn.BatteryTest,itertools.product(['RF'],[[0]],['pos'],enumerate(inoutpairs)))

In [ ]:
with Pool(8) as p:
    p.starmap(agmlearn.BatteryTest,itertools.product(['GBM'],[[0]],['pos'],enumerate(inoutpairs)))

In [ ]:
with Pool(8) as p:
    p.starmap(agmlearn.BatteryTest,itertools.product(['gbm2'],[[0]],['pos'],enumerate(inoutpairs)))

## Train Leave-One-Out Position classifiers on Practice session

In [ ]:
PPinoutpairs = dataloader.GetOrLoadLeaveOneOuts('pracpos_leaveoneout.pkl',PPfea)

In [ ]:
for k in range(3,10):
    with Pool(8) as p:
        p.starmap(agmlearn.BatteryTest,itertools.product(['kNN'],[[k]],['p_pp'],enumerate(agmlearn.NormalizeList(PPinoutpairs))))

In [ ]:
from glob import glob
for k in range(3,10):
    files = glob(f'agm_util/predcache/*knn[[]{k}[]]*p_pp*')
    corr = []
    for f in files:
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'{k}NN: average correct IDs over participants:{np.mean(corr)}')

In [ ]:
with Pool(10) as p:
    p.starmap(agmlearn.BatteryTest,itertools.product(['RF'],[[0]],['p_pp'],enumerate(PPinoutpairs)))

In [ ]:
with Pool(10) as p:
    p.starmap(agmlearn.BatteryTest,itertools.product(['GBM'],[[0]],['p_pp'],enumerate(PPinoutpairs)))

## View Leave-One-Out prediction accuracy on Position.

In [ ]:
from glob import glob
for k in range(3,10):
    files = glob(f'agm_util/predcache/*knn[[]{k}[]]*Normpos*')
    corr = []
    for f in files:
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'{k}NN: average correct IDs over participants:{np.mean(corr)}')

In [ ]:
for i in range(1):
    corr = []
    for f in glob(f'agm_util/predcache/*RF*pos.csv'):
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'RF: average correct IDs over participants: {np.mean(corr)}')

In [ ]:
for i in range(1):
    corr = []
    for f in glob(f'agm_util/predcache/*GBM*pos.csv'):
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'GBM: average correct IDs over participants: {np.mean(corr)}')

In [ ]:
#create plot:
def getcorr(s):
    corr = []
    for f in glob(f'agm_util/predcache/{s}'):
        evaluated = agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID'])
        corr.append(evaluated.Correct.mean())
    return corr

LPcorr = pd.DataFrame()
LPcorr['knn']= getcorr('*knn[[]3[]]*Normpos*')
LPcorr['RF'] = getcorr('*RF*pos.csv')
LPcorr['GBM'] = getcorr('*GBM*pos.csv')
LPcorr['gbm2'] = getcorr('*gbm2*pos.csv')

LPcorr.to_csv('LPcorrectPredictionAverages.csv',index=None)

## View Practice-session prediction accuracy on Position

In [ ]:
knnclf = agmlearn.MakeClassifier('kNN',LPfea,[3])


In [ ]:
knnPracPred = agmlearn.GetOrLoadPrediction(knnclf,PPfea,'knn3PracticePos.csv')
#agmlearn.LoadPrediction('knn3PracticePos.csv')
agmlearn.EvaluatePrediction(knnPracPred,groups=['PID']).Correct.mean()

In [ ]:
knnPracPred = agmlearn.LoadPrediction('knn3PracticePos.csv')

In [ ]:
agmlearn.EvaluatePrediction(knnPracPred).Correct.groupby('subphaseID').sum().max()/60

In [ ]:
rfclf = agmlearn.MakeClassifier('RF',LPfea)
rfPracPred = agmlearn.GetOrLoadPrediction(rfclf,PPfea,'RFPracticePos.csv')

In [ ]:
agmlearn.EvaluatePrediction(rfPracPred,groups=['PID']).Correct.mean()

In [ ]:
rfPracPred = agmlearn.LoadPrediction('RFPracticePos.csv')

In [ ]:
agmlearn.EvaluatePrediction(rfPracPred).Correct.groupby('subphaseID').sum().max()/60

In [ ]:
gbmclf = agmlearn.MakeClassifier('GBM',LPfea)
gbmPracPred = agmlearn.GetOrLoadPrediction(gbmclf,PPfea,'GBMPracticePos.csv')

In [ ]:
agmlearn.EvaluatePrediction(gbmPracPred,groups=['PID']).Correct.mean()

In [ ]:
gbmPracPred = agmlearn.LoadPrediction('GBMPracticePos.csv')

In [ ]:
agmlearn.EvaluatePrediction(gbmPracPred).Correct.groupby('subphaseID').sum().max()/60

In [ ]:
pickle.dump((knnPracPred,rfPracPred,gbmPracPred),open('Pickles/PracticeSessionClassifiers.pkl','wb'))

## Backwards classifier

In [ ]:
from glob import glob
for k in range(3,10):
    files = glob(f'agm_util/predcache/*knn[[]{k}[]]*p_pp*')
    corr = []
    for f in files:
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'{k}NN: average correct IDs over participants:{np.mean(corr)}')
for i in range(1):
    corr = []
    for f in glob(f'agm_util/predcache/*RF*p_pp.csv'):
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'RF: average correct IDs over participants: {np.mean(corr)}')
for i in range(1):
    corr = []
    for f in glob(f'agm_util/predcache/*GBM*p_pp.csv'):
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'GBM: average correct IDs over participants: {np.mean(corr)}')

In [ ]:
def getcorr(s):
    corr = []
    for f in glob(f'agm_util/predcache/{s}'):
        evaluated = agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID'])
        corr.append(evaluated.Correct.mean())
    return corr

LPcorr = pd.DataFrame()
LPcorr['knn']= getcorr('*knn[[]3[]]*p_pp*')
LPcorr['RF'] = getcorr('*RF*p_pp.csv')
LPcorr['GBM'] = getcorr('*GBM*p_pp.csv')
#LPcorr['gbm2'] = getcorr('*gbm2*pos.csv')

LPcorr.to_csv('PPcorrectPredictionAverages.csv',index=None)

In [ ]:
bknnclf = agmlearn.MakeClassifier('kNN',PPfea,[3])

In [ ]:
bknnPracPred = agmlearn.GetOrLoadPrediction(bknnclf,LPfea,'knn3BackwardsPos.csv')
#agmlearn.LoadPrediction('knn3BackwardsPos.csv')
agmlearn.EvaluatePrediction(bknnPracPred,groups=['PID']).Correct.mean()

In [ ]:
brfclf = agmlearn.MakeClassifier('RF',PPfea)

In [ ]:
brfPracPred = agmlearn.GetOrLoadPrediction(brfclf,LPfea,'rfBackwardsPos.csv')
agmlearn.EvaluatePrediction(brfPracPred,groups=['PID']).Correct.mean()

In [ ]:
bgbmclf = agmlearn.MakeClassifier('GBM',PPfea)

In [ ]:
#bgbmPracPred = agmlearn.GetOrLoadPrediction(bgbmclf,LPfea,'gbmBackwardsPos.csv')
bgbmPracPred = agmlearn.LoadPrediction('gbmBackwardsPos.csv')
agmlearn.EvaluatePrediction(bgbmPracPred,groups=['PID']).Correct.mean()

In [ ]:
print('GBM avg',agmlearn.EvaluatePrediction(bgbmPracPred).Correct.mean())
print('GBM min',agmlearn.EvaluatePrediction(bgbmPracPred).Correct.groupby('subphaseID').sum().min()/60)
print('GBM max',agmlearn.EvaluatePrediction(bgbmPracPred).Correct.groupby('subphaseID').sum().max()/60)
print('kNN avg',agmlearn.EvaluatePrediction(bknnPracPred).Correct.mean())
print('kNN min',agmlearn.EvaluatePrediction(bknnPracPred).Correct.groupby('subphaseID').sum().min()/60)
print('kNN max',agmlearn.EvaluatePrediction(bknnPracPred).Correct.groupby('subphaseID').sum().max()/60)
print('RF avg',agmlearn.EvaluatePrediction(brfPracPred).Correct.mean())
print('RF min',agmlearn.EvaluatePrediction(brfPracPred).Correct.groupby('subphaseID').sum().min()/60)
print('RF max',agmlearn.EvaluatePrediction(brfPracPred).Correct.groupby('subphaseID').sum().max()/60)

## Train Leave-One-Out velocity classifiers.

In [ ]:
#Vinoutpairs = dataloader.GetOrLoadLeaveOneOuts('velpairs.pkl',LVfea)
Vinoutpairs = dataloader.GetOrLoadLeaveOneOuts('velpairs.pkl')

In [ ]:
for k in range(3,20):
    with Pool(8) as p:
        p.starmap(agmlearn.BatteryTest,itertools.product(['kNN'],[[k]],['Normvel'],enumerate(agmlearn.NormalizeList(Vinoutpairs))))

In [ ]:
with Pool(8) as p:
    p.starmap(agmlearn.BatteryTest,itertools.product(['RF'],[[0]],['vel'],enumerate(Vinoutpairs)))

In [ ]:
with Pool(8) as p:
    p.starmap(agmlearn.BatteryTest,itertools.product(['GBM'],[[0]],['vel'],enumerate(Vinoutpairs)))

## View Leave-One-Out prediction accuracy on Velocity

In [ ]:
for k in range(3,20):
    files = glob(f'agm_util/predcache/*knn[[]{k}[]]*Normvel*')
    corr = []
    for f in files:
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'{k}NN: average correct IDs over participants:{np.mean(corr)}')

In [ ]:
for i in range(1):
    corr = []
    for f in glob(f'agm_util/predcache/*RF*vel.csv'):
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'RF: average correct IDs over participants: {np.mean(corr)}')
    print(len(corr)

In [ ]:
for i in range(1):
    corr = []
    for f in glob(f'agm_util/predcache/*GBM*vel.csv'):
        corr.append(agmlearn.EvaluatePrediction(pd.read_csv(f),groups=['PID']).Correct.mean())
    print(f'GBM: average correct IDs over participants: {np.mean(corr)}')
    print(len(corr))

In [ ]:
LVcorr = pd.DataFrame()
LVcorr['knn']= getcorr('*knn[[]13[]]*Normvel*')
LVcorr['RF'] = getcorr('*RF*vel.csv')
LVcorr['GBM'] = getcorr('*GBM*vel.csv')
LVcorr.to_csv('LVcorrectPredictionAverages.csv',index=None)

## View Practice-session prediction accuracy on Velocity

In [ ]:
Vknnclf = agmlearn.MakeClassifier('kNN',LVfea,[13])

In [ ]:
VknnPracPred = agmlearn.GetOrLoadPrediction(Vknnclf,PVfea,'knn3PracticeVel.csv')
agmlearn.EvaluatePrediction(VknnPracPred,groups=['PID']).Correct.mean()

In [ ]:
agmlearn.EvaluatePrediction(VknnPracPred).Correct.mean()

In [ ]:
Vrfclf = agmlearn.MakeClassifier('RF',LVfea)
VrfPracPred = agmlearn.GetOrLoadPrediction(Vrfclf,PVfea,'RFPracticeVel.csv')

In [ ]:
agmlearn.EvaluatePrediction(VrfPracPred,groups=['PID']).Correct.mean()

In [ ]:
agmlearn.EvaluatePrediction(VrfPracPred).Correct.mean()

In [ ]:
Vgbmclf = agmlearn.MakeClassifier('GBM',LVfea)
VgbmPracPred = agmlearn.GetOrLoadPrediction(Vgbmclf,PVfea,'GBMPracticeVel.csv')

In [ ]:
agmlearn.EvaluatePrediction(VgbmPracPred,groups=['PID']).Correct.mean()

In [ ]:
agmlearn.EvaluatePrediction(VgbmPracPred).Correct.mean()

In [ ]:
VknnPracPred= agmlearn.LoadPrediction('knn3PracticeVel.csv')
VrfPracPred = agmlearn.LoadPrediction('RFPracticeVel.csv')
VgbmPracPred= agmlearn.LoadPrediction('GBMPracticeVel.csv')

In [ ]:
print('knn',agmlearn.EvaluatePrediction(VknnPracPred).groupby('subphaseID').Correct.sum().min()/60)
print('rf',agmlearn.EvaluatePrediction(VrfPracPred).groupby('subphaseID').Correct.sum().min()/60)
print('gbm',agmlearn.EvaluatePrediction(VgbmPracPred).groupby('subphaseID').Correct.sum().min()/60)

## Visualize change in correctness over time

In [ ]:
#get practice prediction data ready
knnPracPred = agmlearn.LoadPrediction('knn3PracticePos.csv')
rfPracPred = agmlearn.LoadPrediction('RFPracticePos.csv')
gbmPracPred = agmlearn.LoadPrediction('GBMPracticePos.csv')

In [ ]:
#determine participant to visualize
PPcorrectness = pd.DataFrame()
PPcorrectness['kNN'] = agmlearn.EvaluatePrediction(knnPracPred,groups=['PID'])['Correct']
PPcorrectness['RF'] = agmlearn.EvaluatePrediction(rfPracPred,groups=['PID'])['Correct']
PPcorrectness['GBM'] = agmlearn.EvaluatePrediction(gbmPracPred,groups=['PID'])['Correct']
PPcorrectness.loc[(PPcorrectness['GBM']) & (PPcorrectness['kNN']!=True)]

In [ ]:
for ID_to_look_at in PPcorrectness.loc[(PPcorrectness['RF']) & (PPcorrectness['kNN']!=True)].index:
    PPpred = pd.DataFrame()
    #ID_to_look_at = 18
    PPpred['kNN'] = knnPracPred.loc[knnPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['kNN_cumsum'] = np.cumsum(PPpred['kNN']==ID_to_look_at)
    PPpred['RF'] = rfPracPred.loc[rfPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['RF_cumsum'] = np.cumsum(PPpred['RF']==ID_to_look_at)
    PPpred['GBM'] = gbmPracPred.loc[gbmPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['GBM_cumsum'] = np.cumsum(PPpred['GBM']==ID_to_look_at)
    PPpred['totalVotes'] = np.arange(len(PPpred),dtype=int)+1

    for i in range(60):
        if i!=ID_to_look_at:
            PPpred[f'k_c_w{i}'] = np.cumsum(knnPracPred.loc[knnPracPred['PID']==ID_to_look_at]['Prediction']==i)
            PPpred[f'r_c_w{i}'] = np.cumsum(rfPracPred.loc[rfPracPred['PID']==ID_to_look_at]['Prediction']==i)
            PPpred[f'g_c_w{i}'] = np.cumsum(gbmPracPred.loc[gbmPracPred['PID']==ID_to_look_at]['Prediction']==i)
    for t in 'krg':
        PPpred[f'{t}_WorstContender'] = PPpred[[f'{t}_c_w{i}' for i in range(60) if i != ID_to_look_at]].max(axis=1)
        PPpred[f'{t}_CurrentCorrect'] = Ppred

    plt.plot(PPpred.totalVotes,PPpred.GBM_cumsum/PPpred.g_WorstContender,label='GBM',color='green')
    plt.plot(PPpred.totalVotes,PPpred.kNN_cumsum/PPpred.k_WorstContender,label='kNN',color='blue')
    plt.plot(PPpred.totalVotes,PPpred.RF_cumsum/PPpred.r_WorstContender,label='RF',color='goldenrod')

    #a = PPpred.plot('totalVotes',['kNN_cumsum','RF_cumsum','GBM_cumsum'])
    plt.plot(np.arange(len(PPpred)),np.ones(len(PPpred)),'--',color='gray',alpha=0.3,)
    #plt.ylim((0,PPpred[['kNN_cumsum','RF_cumsum','GBM_cumsum']].max().max()*1.05))
    plt.ylim((.1,100))
    plt.xlabel('Total identifications')
    plt.ylabel('|Correct ID| / |most voted incorrect ID|')
    plt.yscale('log')
    plt.title('Identification accuracy through practice session')
    plt.legend()
    plt.savefig(f'PracticeIDPlots/Position_Practice_PID={ID_to_look_at}.png')
    plt.show()


In [ ]:
CurrCorr = []
for ID_to_look_at in range(60):
    PPpred = pd.DataFrame()
    #ID_to_look_at = 18
    PPpred['kNN'] = knnPracPred.loc[knnPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['kNN_cumsum'] = np.cumsum(PPpred['kNN']==ID_to_look_at)
    PPpred['RF'] = rfPracPred.loc[rfPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['RF_cumsum'] = np.cumsum(PPpred['RF']==ID_to_look_at)
    PPpred['GBM'] = gbmPracPred.loc[gbmPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['GBM_cumsum'] = np.cumsum(PPpred['GBM']==ID_to_look_at)
    PPpred['totalVotes'] = np.arange(len(PPpred),dtype=int)+1

    for i in range(60):
        if i!=ID_to_look_at:
            PPpred[f'k_c_w{i}'] = np.cumsum(knnPracPred.loc[knnPracPred['PID']==ID_to_look_at]['Prediction']==i)
            PPpred[f'r_c_w{i}'] = np.cumsum(rfPracPred.loc[rfPracPred['PID']==ID_to_look_at]['Prediction']==i)
            PPpred[f'g_c_w{i}'] = np.cumsum(gbmPracPred.loc[gbmPracPred['PID']==ID_to_look_at]['Prediction']==i)
    for t,f in zip('krg',['kNN','RF','GBM']):
        PPpred[f'{t}_WorstContender'] = PPpred[[f'{t}_c_w{i}' for i in range(60) if i != ID_to_look_at]].max(axis=1)
        PPpred[f'{t}_CurrentCorrect'] = PPpred[f'{f}_cumsum']>PPpred[f'{t}_WorstContender']
    
    CurrCorr.append(PPpred[[f'{t}_CurrentCorrect' for t in 'krg']])
    CurrCorr[-1].columns = [f'{t}_CurrentCorrect_{ID_to_look_at}' for t in 'krg']

max([len(x) for x in CurrCorr])


In [ ]:
CurrCorr[0]

In [ ]:
a = []
for c in CurrCorr:
    a.append(c.reset_index())

sumout = pd.DataFrame()
for t in 'krg':
    sumout[t] = pd.concat([x[f'{t}_CurrentCorrect_{i}'] for i,x in enumerate(a)],axis=1).fillna(method='ffill').sum(axis=1)
sumout
sumout.to_csv('IDsbytime_pos.csv')

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(range(len(sumout)),sumout['g'],label='GBM',color='green')
plt.plot(range(len(sumout)),sumout['k'],label='kNN',color='blue')
plt.plot(range(len(sumout)),sumout['r'],label='RF',color='goldenrod')

#a = PPpred.plot('totalVotes',['kNN_cumsum','RF_cumsum','GBM_cumsum'])
#plt.plot(np.arange(len(PPpred)),np.ones(len(PPpred)),'--',color='gray',alpha=0.3,)
#plt.ylim((0,PPpred[['kNN_cumsum','RF_cumsum','GBM_cumsum']].max().max()*1.05))
#plt.ylim((.1,100))
plt.xlabel('Seconds')
plt.ylabel('Correctly identified participants')
#plt.yscale('log')
plt.title('Identification accuracy through practice session')
plt.legend()
plt.savefig(f'IdentificationAccThroughSession_position.png')
plt.show()


In [ ]:
VCurrCorr = []
for ID_to_look_at in range(60):
    PPpred = pd.DataFrame()
    #ID_to_look_at = 18
    PPpred['kNN'] = VknnPracPred.loc[VknnPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['kNN_cumsum'] = np.cumsum(PPpred['kNN']==ID_to_look_at)
    PPpred['RF'] = VrfPracPred.loc[VrfPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['RF_cumsum'] = np.cumsum(PPpred['RF']==ID_to_look_at)
    PPpred['GBM'] = VgbmPracPred.loc[VgbmPracPred['PID']==ID_to_look_at]['Prediction']
    PPpred['GBM_cumsum'] = np.cumsum(PPpred['GBM']==ID_to_look_at)
    PPpred['totalVotes'] = np.arange(len(PPpred),dtype=int)+1

    for i in range(60):
        if i!=ID_to_look_at:
            PPpred[f'k_c_w{i}'] = np.cumsum(VknnPracPred.loc[VknnPracPred['PID']==ID_to_look_at]['Prediction']==i)
            PPpred[f'r_c_w{i}'] = np.cumsum(VrfPracPred.loc[VrfPracPred['PID']==ID_to_look_at]['Prediction']==i)
            PPpred[f'g_c_w{i}'] = np.cumsum(VgbmPracPred.loc[VgbmPracPred['PID']==ID_to_look_at]['Prediction']==i)
    for t,f in zip('krg',['kNN','RF','GBM']):
        PPpred[f'{t}_WorstContender'] = PPpred[[f'{t}_c_w{i}' for i in range(60) if i != ID_to_look_at]].max(axis=1)
        PPpred[f'{t}_CurrentCorrect'] = PPpred[f'{f}_cumsum']>PPpred[f'{t}_WorstContender']
    
    VCurrCorr.append(PPpred[[f'{t}_CurrentCorrect' for t in 'krg']])
    VCurrCorr[-1].columns = [f'{t}_CurrentCorrect_{ID_to_look_at}' for t in 'krg']

max([len(x) for x in VCurrCorr])


In [ ]:
v = []
for c in VCurrCorr:
    v.append(c.reset_index())

sumout = pd.DataFrame()
for t in 'krg':
    sumout[t] = pd.concat([x[f'{t}_CurrentCorrect_{i}'] for i,x in enumerate(v)],axis=1).fillna(method='ffill').sum(axis=1)
sumout
sumout.to_csv('IDsbytime_vel.csv')

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(range(len(sumout)),sumout['g'],label='GBM',color='green')
plt.plot(range(len(sumout)),sumout['k'],label='kNN',color='blue')
plt.plot(range(len(sumout)),sumout['r'],label='RF',color='goldenrod')

#a = PPpred.plot('totalVotes',['kNN_cumsum','RF_cumsum','GBM_cumsum'])
#plt.plot(np.arange(len(PPpred)),np.ones(len(PPpred)),'--',color='gray',alpha=0.3,)
#plt.ylim((0,PPpred[['kNN_cumsum','RF_cumsum','GBM_cumsum']].max().max()*1.05))
#plt.ylim((.1,100))
plt.xlabel('Seconds')
plt.ylabel('Correctly identified participants')
#plt.yscale('log')
plt.title('Identification accuracy through practice session')
plt.legend()
plt.savefig(f'IdentificationAccThroughSession_velocity.png')
plt.show()

In [ ]:
currcorrfinal = pd.DataFrame
for t in 'krg':
    currcorrfinal[t] = [pd.concat(CurrCorr[f{t}_CurrentCorrect_{i}],ignore_index=True,axis=1).fillna(method='ffill').sum(axis=1)
